In [2]:
import pandas as pd;
from sqlalchemy import create_engine
import mysql.connector

In [3]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='Basel00272727$',
    database='sakila'
)

In [14]:
#step#1 connection string
username='root'
password='Basel00272727$'
host='127.0.0.1'
port='3306'
databaseName='sakila'

#connection string format
connectionStr=f"mysql+pymysql://{username}:{password}@{host}:{port}/{databaseName}"

In [15]:
engine=create_engine(connectionStr)

In [4]:
import pandas as pd

# تحميل الجداول
payment = pd.read_sql("SELECT payment_id, staff_id, rental_id, amount, payment_date FROM payment", conn)
dim_date = pd.read_sql("SELECT Date_ID, Full_Date FROM dim_date", conn)
dim_staff = pd.read_sql("SELECT staff_id FROM dim_staff", conn)
dim_rent = pd.read_sql("SELECT rental_id FROM dim_rent", conn)

# تحويل Full_Date إلى datetime
dim_date['Full_Date'] = pd.to_datetime(dim_date['Full_Date'])

# فلترة القيم غير الموجودة في dim_staff و dim_rent
payment = payment[
    payment['staff_id'].isin(dim_staff['staff_id']) & 
    payment['rental_id'].isin(dim_rent['rental_id'])
]

# استخراج بداية الشهر
payment['month_start'] = payment['payment_date'].dt.to_period('M').dt.to_timestamp()

# دمج مع dim_date للحصول على date_id
payment = payment.merge(dim_date, left_on='month_start', right_on='Full_Date', how='inner')

# التجميع حسب الشهر، الموظف، والإيجار
df = payment.groupby(['Date_ID', 'staff_id', 'rental_id'], as_index=False)['amount'].sum()

# إعادة التسمية
df = df.rename(columns={
    'Date_ID': 'date_id',
    'staff_id': 'staff_id',
    'rental_id': 'rental_id',
    'amount': 'amount'
})



# المعاينة
print(df.head(100))


C:\Users\basil\AppData\Local\Temp\ipykernel_21600\3851720001.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payment = pd.read_sql("SELECT payment_id, staff_id, rental_id, amount, payment_date FROM payment", conn)
C:\Users\basil\AppData\Local\Temp\ipykernel_21600\3851720001.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_date = pd.read_sql("SELECT Date_ID, Full_Date FROM dim_date", conn)
C:\Users\basil\AppData\Local\Temp\ipykernel_21600\3851720001.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_staff = p

    date_id  staff_id  rental_id  amount
0       121         1          1    2.99
1       121         1          4    4.99
2       121         1          5    6.99
3       121         1          6    0.99
4       121         1          9    4.99
..      ...       ...        ...     ...
95      121         1        184    0.99
96      121         1        186    4.99
97      121         1        188    4.99
98      121         1        189    4.99
99      121         1        192    2.99

[100 rows x 4 columns]


In [19]:
df.to_sql(name='fact_payment', con=engine, if_exists='append', index=False)

15861

In [26]:
import pandas as pd
import plotly.express as px

# نفترض أن لديك df يحتوي على:
# columns: ['staff_id', 'rental_id', 'date_id', 'amount']
# وأنك تريد عرض تطور المبالغ لكل موظف حسب الشهر

# إذا كان لديك dim_date، اربطها للحصول على التاريخ الحقيقي
dim_date = pd.read_sql("SELECT Date_ID, Full_Date FROM dim_date", conn)
dim_date['Full_Date'] = pd.to_datetime(dim_date['Full_Date'])

# دمج df مع dim_date للحصول على التاريخ الكامل
df = df.merge(dim_date, left_on='date_id', right_on='Date_ID', how='left')

# رسم خطي تفاعلي لكل موظف عبر الزمن
fig = px.line(
    df,
    x='Full_Date',
    y='amount',
    color='staff_id',
    title='إجمالي المبالغ المدفوعة لكل موظف عبر الشهور',
    markers=True,
    labels={'Full_Date': 'التاريخ', 'amount': 'إجمالي المبلغ', 'staff_id': 'الموظف'}
)

fig.update_layout(
    xaxis_title="الشهر",
    yaxis_title="المبلغ (USD)",
    legend_title="الموظف",
    template="plotly_dark",  # يمكنك استخدام "plotly_white" أو "seaborn"
)

fig.show()


C:\Users\basil\AppData\Local\Temp\ipykernel_13048\1779853606.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [29]:
import pandas as pd
import plotly.express as px

# دمج df مع dim_date للحصول على Full_Date
df_merged = df.merge(dim_date[['Date_ID', 'Full_Date']], left_on='date_id', right_on='Date_ID', how='left')

# تحويل Full_Date إلى اسم الشهر
df_merged['month'] = df_merged['Full_Date'].dt.strftime('%b-%Y')

# فلترة الموظفين 1 و 2 فقط
df_filtered = df_merged[df_merged['staff_id'].isin([1, 2])]

# رسم الأعمدة بجانب بعض وتحديد ألوان الموظفين
fig = px.bar(
    df_filtered,
    x='month',
    y='amount',
    color='staff_id',
    barmode='group',
    color_discrete_map={
        1: 'red',
        2: 'blue'
    },
    labels={'month': 'الشهر', 'amount': 'إجمالي المبيعات', 'staff_id': 'الموظف'},
    title='📊 مقارنة المبيعات الشهرية بين الموظف 1 (أحمر) والموظف 2 (أزرق)'
)

# تحسين العرض
fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False),
    yaxis=dict(title='إجمالي المبيعات', showgrid=True)
)

fig.show()


In [7]:
import pandas as pd
import plotly.express as px

# ✅ تأكد أن df يحتوي على store_id و Full_Date
df = df.merge(dim_staff[['staff_id', 'store_id']], on='staff_id', how='left')
df = df.merge(dim_date[['Date_ID', 'Full_Date']], left_on='date_id', right_on='Date_ID', how='left')
df['month'] = df['Full_Date'].dt.strftime('%b-%Y')

# ✅ تجميع البيانات حسب الشهر والمتجر
monthly_sales = df.groupby(['month', 'store_id'], as_index=False)['amount'].sum()

# ✅ إجمالي المبيعات لكل شهر
monthly_totals = monthly_sales.groupby('month')['amount'].transform('sum')

# ✅ حساب النسبة المئوية لكل متجر من مبيعات الشهر
monthly_sales['percentage'] = (monthly_sales['amount'] / monthly_totals) * 100

# ✅ رسم النسبة المئوية للمبيعات لكل متجر شهريًا
fig = px.bar(
    monthly_sales,
    x='month',
    y='percentage',
    color='store_id',
    barmode='stack',
    labels={
        'month': 'الشهر',
        'percentage': 'النسبة المئوية من المبيعات',
        'store_id': 'المتجر'
    },
    title='📊 نسبة مبيعات كل متجر من إجمالي مبيعات الشهر'
)

fig.update_layout(
    xaxis_tickangle=-45,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False),
    yaxis=dict(title='النسبة المئوية (%)', showgrid=True),
    yaxis_ticksuffix='%'
)

fig.show()


KeyError: "['store_id'] not in index"